# Markov-Switching Regime Model

This notebook implements the Markov-switching autoregressive model for regime detection, which explicitly models temporal dynamics through a transition matrix.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from statsmodels.tsa.regime_switching.markov_autoregression import MarkovAutoregression

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

## Load Data

In [ ]:
def transform_series(x, tcode):
    if tcode == 1: return x
    elif tcode == 2: return x.diff()
    elif tcode == 3: return x.diff().diff()
    elif tcode == 4: return np.log(x)
    elif tcode == 5: return np.log(x).diff()
    elif tcode == 6: return np.log(x).diff().diff()
    elif tcode == 7: return x.pct_change()
    else: raise ValueError(f"Unknown tcode: {tcode}")

def load_data(filepath='../data/macro_dataset.csv', start_date='1962-07-01'):
    data = pd.read_csv(filepath, skiprows=[1], index_col=0)
    data.columns = [c.upper() for c in data.columns]
    data = data.loc[pd.notna(data.index), :]
    data.index = pd.date_range(start="1959-01-01", freq="MS", periods=len(data))
    
    tcodes = pd.read_csv(filepath, nrows=1, index_col=0)
    tcodes.columns = [c.upper() for c in tcodes.columns]
    
    data = data.apply(lambda x: transform_series(x, tcodes[x.name].item()))
    data = data.dropna(axis=1, subset=[pd.Timestamp(start_date)])
    data = data.fillna(method='ffill').dropna()
    data = data[data.index >= start_date]
    
    scaler = StandardScaler()
    data_std = pd.DataFrame(scaler.fit_transform(data), index=data.index, columns=data.columns)
    return data_std

df = load_data()
X = df.values
dates = df.index
T, p = X.shape

print(f"Data: {T} observations x {p} features")
print(f"Sample: {dates[0].strftime('%Y-%m')} to {dates[-1].strftime('%Y-%m')}")

In [ ]:
K = 4 
print(f"Target regimes: K = {K}")

## Dimensionality Reduction via PCA

Markov-switching models are computationally expensive in high dimensions. We reduce to principal components while preserving key variation.

In [ ]:
# PCA for dimension reduction
n_components = 3 

pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X)

print(f"PCA explained variance:")
for i, var in enumerate(pca.explained_variance_ratio_):
    print(f"  PC{i+1}: {100*var:.1f}%")
print(f"  Total: {100*sum(pca.explained_variance_ratio_):.1f}%")

df_pca = pd.DataFrame(X_pca, index=dates, columns=[f'PC{i+1}' for i in range(n_components)])

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 8), sharex=True)

# NBER recession dates for shading
recessions = [
    ('1969-12-01', '1970-11-01'), ('1973-11-01', '1975-03-01'),
    ('1980-01-01', '1980-07-01'), ('1981-07-01', '1982-11-01'),
    ('1990-07-01', '1991-03-01'), ('2001-03-01', '2001-11-01'),
    ('2007-12-01', '2009-06-01'), ('2020-02-01', '2020-04-01'),
]

for i, col in enumerate(df_pca.columns):
    ax = axes[i]
    ax.plot(dates, df_pca[col], linewidth=0.8)
    ax.set_ylabel(f'{col} ({100*pca.explained_variance_ratio_[i]:.1f}%)')
    ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
    
    for start, end in recessions:
        ax.axvspan(pd.Timestamp(start), pd.Timestamp(end), color='gray', alpha=0.3)

axes[-1].set_xlabel('Date')
plt.suptitle('Principal Components with NBER Recession Shading', fontsize=12)
plt.tight_layout()
plt.show()

## Markov-Switching Autoregression

We fit a Markov-switching AR(1) model on PC1 (the dominant factor):

$$y_t = \alpha_{S_t} + \phi y_{t-1} + \sigma_{S_t} \epsilon_t$$

where $S_t \in \{0, 1, ..., K-1\}$ follows a Markov chain with transition matrix $\Pi$.

In [ ]:
# Fit Markov-switching model on PC1
y = df_pca['PC1']

print("Fitting Markov-Switching AR(1) model...")
print("(This may take a minute)")

ms_model = MarkovAutoregression(
    y,
    k_regimes=K,
    order=1,           # AR(1)
    switching_ar=False,  # AR coefficient same across regimes
    switching_variance=True  # Variance differs by regime
)

# Fit with multiple starting points
ms_results = ms_model.fit(search_reps=20, maxiter=500)

print("\nModel fitted successfully!")
print(f"Log-likelihood: {ms_results.llf:.2f}")
print(f"AIC: {ms_results.aic:.2f}")
print(f"BIC: {ms_results.bic:.2f}")

In [ ]:
print(ms_results.summary())

In [ ]:
# Extract regime probabilities
smoothed_probs = ms_results.smoothed_marginal_probabilities

print(f"Smoothed probabilities shape: {smoothed_probs.shape}")
print(f"\nFirst 5 periods:")
print(smoothed_probs.head().round(3))

In [ ]:
ms_soft = smoothed_probs.values
ms_dates = smoothed_probs.index
if len(ms_soft) < T:
    n_missing = T - len(ms_soft)
    padding = np.tile(ms_soft[0], (n_missing, 1))
    ms_soft = np.vstack([padding, ms_soft])
    print(f"Padded {n_missing} observations at start")

ms_hard = np.argmax(ms_soft, axis=1)

print(f"\nFinal shapes:")
print(f"  Soft assignments: {ms_soft.shape}")
print(f"  Hard assignments: {ms_hard.shape}")

In [ ]:
# Regime distribution
print("Regime distribution (Markov-Switching):")
unique, counts = np.unique(ms_hard, return_counts=True)
for r, c in zip(unique, counts):
    print(f"  Regime {r}: {c} months ({100*c/T:.1f}%)")

## Transition Matrix

In [ ]:
def estimate_transition_matrix(hard_labels, K):
    """Estimate transition matrix from hard regime labels"""
    trans_counts = np.zeros((K, K))
    for t in range(len(hard_labels) - 1):
        trans_counts[hard_labels[t], hard_labels[t+1]] += 1
    
    row_sums = trans_counts.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1
    return trans_counts / row_sums

trans_matrix = estimate_transition_matrix(ms_hard, K)

print("Estimated Transition Matrix:")
print(pd.DataFrame(trans_matrix, 
                   index=[f'From R{i}' for i in range(K)],
                   columns=[f'To R{i}' for i in range(K)]).round(3))

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(trans_matrix, annot=True, fmt='.3f', cmap='Blues',
            xticklabels=[f'To R{i}' for i in range(K)],
            yticklabels=[f'From R{i}' for i in range(K)])
plt.title('Markov-Switching Transition Matrix')
plt.tight_layout()
plt.show()

print(f"\nSelf-transition probabilities (diagonal):")
for i in range(K):
    print(f"  Regime {i}: {trans_matrix[i,i]:.3f}")
print(f"  Average: {np.mean(np.diag(trans_matrix)):.3f}")

In [ ]:
print("Expected regime durations (months):")
for i in range(K):
    if trans_matrix[i,i] < 1:
        expected_duration = 1 / (1 - trans_matrix[i,i])
        print(f"  Regime {i}: {expected_duration:.1f} months")
    else:
        print(f"  Regime {i}: Absorbing state")

## Visualize Regime Probabilities

In [ ]:
colors = plt.cm.Set1(np.linspace(0, 1, K))

fig, ax = plt.subplots(figsize=(14, 6))

ax.stackplot(dates, ms_soft.T, colors=colors, alpha=0.8,
             labels=[f'Regime {i}' for i in range(K)])

for start, end in recessions:
    ax.axvspan(pd.Timestamp(start), pd.Timestamp(end), 
               color='gray', alpha=0.3, zorder=10)

ax.set_xlabel('Date')
ax.set_ylabel('Regime Probability')
ax.set_ylim(0, 1)
ax.legend(loc='upper right', ncol=K)
ax.set_title('Markov-Switching Smoothed Regime Probabilities\n(Gray = NBER recessions)')
plt.tight_layout()
plt.show()

In [ ]:
# Hard regime assignments over time
fig, ax = plt.subplots(figsize=(14, 3))

for t in range(len(dates)):
    ax.axvspan(dates[t], dates[min(t+1, len(dates)-1)], 
               color=colors[ms_hard[t]], alpha=0.8)

# Add recession markers
for start, end in recessions:
    ax.axvspan(pd.Timestamp(start), pd.Timestamp(end), 
               color='none', edgecolor='black', linewidth=2, hatch='//')

ax.set_xlabel('Date')
ax.set_yticks([])
ax.set_title('Markov-Switching Hard Regime Assignments (hatched = NBER recessions)')

legend_elements = [plt.Rectangle((0,0),1,1, color=colors[i], label=f'Regime {i}') 
                   for i in range(K)]
ax.legend(handles=legend_elements, loc='upper right', ncol=K)
plt.tight_layout()
plt.show()

## Regime Characteristics

In [ ]:
print("Regime Characteristics (Mean PC values):")
print("="*50)

for regime in range(K):
    mask = ms_hard == regime
    n_periods = mask.sum()
    
    print(f"\nRegime {regime} ({n_periods} periods):")
    for i, col in enumerate(df_pca.columns):
        mean_val = df_pca.loc[dates[mask], col].mean()
        print(f"  {col}: {mean_val:.3f}")

In [ ]:
transitions = np.sum(ms_hard[1:] != ms_hard[:-1])
trans_freq = transitions / (T - 1)

durations = []
current_dur = 1
for t in range(1, len(ms_hard)):
    if ms_hard[t] == ms_hard[t-1]:
        current_dur += 1
    else:
        durations.append(current_dur)
        current_dur = 1
durations.append(current_dur)

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(durations, bins=range(1, max(durations)+5, 2), edgecolor='black', alpha=0.7)
plt.axvline(np.mean(durations), color='red', linestyle='--', 
            label=f'Mean: {np.mean(durations):.1f}')
plt.axvline(np.median(durations), color='orange', linestyle='--',
            label=f'Median: {np.median(durations):.1f}')
plt.xlabel('Duration (months)')
plt.ylabel('Frequency')
plt.title('Distribution of Regime Durations (Markov-Switching)')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
ms_results_dict = {
    'soft': ms_soft,
    'hard': ms_hard,
    'transition_matrix': trans_matrix,
    'dates': dates,
}